### Importing required libraries.

The data is collected from yahoo finance on a daily basis which can be altered depending on the requirement . 

In [1]:
#Installing Yahoo Finance
!pip install yfinance
!pip install pandas_datareader
!pip install scikeras
!pip install imbalanced-learn
!pip install xgboost
!pip install TA-Lib

  Using cached TA-Lib-0.4.25.tar.gz (271 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
Failed to build TA-Lib


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\henry\anaconda3\envs\dev1\python.exe' 'C:\Users\henry\anaconda3\envs\dev1\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py' build_wheel 'C:\Users\henry\AppData\Local\Temp\tmpjl4m0k9z'
       cwd: C:\Users\henry\AppData\Local\Temp\pip-install-lks_z20a\ta-lib_f950ad26c1774d9a9c0f8e0dcc5717e2
  Complete output (21 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-37
  creating build\lib.win-amd64-cpython-37\talib
  copying talib\abstract.py -> build\lib.win-amd64-cpython-37\talib
  copying talib\deprecated.py -> build\lib.win-amd64-cpython-37\talib
  copying talib\stream.py -> build\lib.win-amd64-cpython-37\talib
  copying talib\test_abstract.py -> build\lib.win-amd64-cpython-37\talib
  copying talib\test_data.py -> build\lib.win-amd64-cpython-37\talib
  copying talib\test_func.py -> build\lib.win-amd64-cpython-37\talib
  copying talib

In [2]:
import pandas as pd
import time, datetime
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import os
import yfinance as yf
import pandas_datareader.data as pdr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers import SGD
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE 
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from itertools import product
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

### Importing Data from Yahoo Finance.

In [3]:
#Collecting desired ticker from user
ticker = input("Please enter your desired ticker")
start_time = datetime.datetime(1980,1,1)
end_time = datetime.datetime.now().date()

Please enter your desired ticker NDAQ


In [23]:
def get_data(ticker):
    df = pdr.get_data_yahoo(ticker, start=start_time, end=end_time)         
    return df

In [24]:
def compute_ema(df):
    df['EMA50'] = df['Close'].ewm(span=50, adjust=False).mean()
    #Setting adjust to False to specify that recursive calculation mode is required.
    df['EMA100'] = df['Close'].ewm(span=100, adjust=False).mean()
    #df['EMA150'] = EMA(df['Close'].values, timeperiod=150)
   # df['EMA200'] = EMA(df['Close'].values, timeperiod=200)
    return df


In [25]:
#Computing the MACD indicator
#df =computeMACD(df, 12, 26, 9)
#The values can be altered along with the model. 
#The values are normalised between -1 to 1 which is not suitable for the current model
#As we are looking for the intersection of price and MACD.

### Creating the feature variables.

In [26]:
def calculated_features(df):
    df['aboveEMA50'] = np.where(df['Close'] > df['EMA50'], 1, 0)
    df['aboveEMA100'] = np.where(df['Close'] > df['EMA100'], 1, 0)
    return df

In [27]:
# create a new column in the DF based on the conditions
def defined_conditions(df):
    # List of conditions
    condition = [(df['aboveEMA50'] == df['aboveEMA100']) & (df['aboveEMA50'] ==1),
                 (df['aboveEMA50'] == df['aboveEMA100']) & (df['aboveEMA50'] ==0),
                 (df['aboveEMA50'] != df['aboveEMA100'])]
    # List of values to return
    choice  = [1,0,2]

    df['Buy/Sell'] = np.select(condition, choice, "ERROR")
    
    return df
#Considering it as a buy if the price is over 50 ema and 100 ema.
#Considering it as a Sell if the price is under 50 ema and 100 ema. 
#Further analysis is required if the price is over 50 ema and under ema 100 or vice versa.

### Creating DF for multiple stocks and Indices.

In [28]:
# Requesting the stock Data from the user. 
#tickers = ['AAPL','GOOG','AMD','AMZN']

In [29]:


#Calling the get_data function definition. 
df = get_data(ticker)
df = df.reset_index()
df = df.set_index('Date')
#Calculating EMA 50 and 100
df = compute_ema(df)
#Calculating if the signal is to buy or sell.
df = calculated_features(df)
# Rules for placind a trade.
df = defined_conditions(df)


### Seperating Labels and Features from the Data frame and splitting the data

In [30]:
X = df.drop(columns =['Buy/Sell', 'High', 'Low', 'Open', 'Adj Close', 'aboveEMA50', 'aboveEMA100'])
#Reviewing features Data Frame.
X.head(5)

,Close,Volume,EMA50,EMA100
Date,,,,
2002-07-01,5.000000,3900.0,5.000000,5.000000
2002-07-02,5.043333,21000.0,5.001699,5.000858
2002-07-03,5.000000,30600.0,5.001633,5.000841
2002-07-05,5.000000,1500.0,5.001569,5.000824
2002-07-08,4.666667,900.0,4.988435,4.994207


In [31]:
y = df['Buy/Sell']
#Reviewing Label.
y.head(5)
#y.value_counts()
## Found class Imbalances in the data set. 
# Suggested to use SMOTE Upsampling to balane the class.


Date
2002-07-01    0
2002-07-02    1
2002-07-03    0
2002-07-05    0
2002-07-08    0
Name: Buy/Sell, dtype: object

In [32]:
# Split the data into training and testing datasets
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state =1)

### Passing the Data to NN to establish a baseline accuracy.

#### Normalising the data using Standard Scaler

In [33]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Scale both the training and testing data from the features dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# encoding class labels as integers
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)

#### Adding Layers to Neural Network

In [34]:
def create_model():
	# create model
	model = Sequential()
	model.add(Dropout(0.2, input_shape=(4,)))
	model.add(Dense(10, activation='relu', kernel_constraint=MaxNorm(3)))
	model.add(Dense(5, activation='relu', kernel_constraint=MaxNorm(3)))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	sgd = SGD(learning_rate=0.1, momentum=0.9)
	model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
	return model
 

In [35]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(model=create_model, epochs=30, batch_size=1, verbose=0)))
pipeline = Pipeline(estimators)

In [36]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

In [37]:
results = cross_val_score(pipeline, X, y, cv=kfold)

In [38]:
print("Visible: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

### Additional tune up and Learning rate and Epochs needed. 

Visible: 31.98% (0.04%)


### Creating Ensemble

In [39]:
from sklearn.metrics import log_loss
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()
#clf4 = XGBClassifier()
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],voting='hard')


eclf.fit(X_train, y_train)
 
# predicting the output on the test dataset
pred_final = eclf.predict(X_test)
 
# printing log loss between actual and predicted value
print(accuracy_score(y_test, pred_final))

0.6303501945525292


In [40]:
for clf, label in zip([clf1, clf2, clf3,eclf], ['Logistic Regression', 'Random Forest', 'Naive Bayes', 'Ensemble']):
    scores = cross_val_score(clf, X_test, y_test, scoring='accuracy', cv=5)
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.58 (+/- 0.00) [Logistic Regression]
Accuracy: 0.86 (+/- 0.01) [Random Forest]
Accuracy: 0.62 (+/- 0.02) [Naive Bayes]
Accuracy: 0.65 (+/- 0.02) [Ensemble]


In [22]:
## Models to be tuned up for better accuracy. 